In [2]:
from __future__ import print_function
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint


ModuleNotFoundError: No module named 'swagger_client'

In [3]:

# Configure API key authorization: ApiKeyAuth
configuration = swagger_client.Configuration()
configuration.api_key['key'] = 'YOUR_API_KEY'
# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
# configuration.api_key_prefix['key'] = 'Bearer'

# create an instance of the API class
api_instance = swagger_client.APIsApi(swagger_client.ApiClient(configuration))
q = 'Moscow' # Указываем название города Москва
dt = '2024-04-18' # Указываем сегодняшнюю дату
lang = 'ru' # Язык ответа - русский

try:
    # Получаем текущую погоду
    api_response = api_instance.current_weather(q, lang=lang)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling APIsApi->current_weather: %s\n" % e)

NameError: name 'swagger_client' is not defined